<a href="https://colab.research.google.com/github/zooodung/Face_Image_Emotion_Classification/blob/JS/FP_ViTb16_Test_Data_Result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('/content/drive/MyDrive/Project/ViT_b16')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/lambda_layer.py:327: UserWarning: vit_keras.vit is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/lambda_layer.py:327: UserWarning: vit_keras.layers is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


In [ ]:
import zipfile
import os

def unzip_file(zip_file_path, extract_dir):
    # 압축 파일 열기
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # 압축 해제
        zip_ref.extractall(extract_dir)

# 압축 파일 경로와 추출할 디렉토리 경로 지정
zip_file_path = '/content/drive/MyDrive/Project/Data/test_img.Zip'
extract_dir = '/content/test_img_p'

# 압축 파일 풀기
unzip_file(zip_file_path, extract_dir)

In [ ]:
!pip install vit_keras
!pip install tensorflow-addons
! pip install face_recognition
! pip install imgaug
! pip install dlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=3350b6d3e9943033c7b68d16b54e6a3d21c44c970d1bc0383d3bda5328f16f37
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import numpy as np
import imgaug as ia
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import imgaug.augmenters as iaa
import face_recognition

In [ ]:
#faceposeaugument는 pytorch 기반이라 tensorflow와 호환되는 imgaug 라이브러리 사용
def augment_face_pose(image, augmenter):
    # RGB 이미지로 변환
    image_rgb = image.astype('uint8')[:, :, ::-1]

    # 특징점 추출
    landmarks = face_recognition.face_landmarks(image_rgb)
    if not landmarks:  # 얼굴이 검출되지 않으면 증강 X
        return image

    # 랜드마크 좌표 추출
    keypoints = [ia.Keypoint(x=landmark[0], y=landmark[1]) for landmark in landmarks[0]['chin']]

    # 랜드마크 기반 증강
    augmented_image, _ = augmenter(image=image, keypoints=ia.KeypointsOnImage(keypoints, shape=image.shape))
    return augmented_image

augmenter = iaa.Affine(
    scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},  # 얼굴 크기 조절 (80% ~ 120%)
    translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},  # 얼굴 위치 이동 (-20% ~ 20%)
    rotate=(-25, 25),  # 얼굴 회전 (-25도 ~ 25도)
    shear=(-10, 10),  # 얼굴 왜곡 (-10도 ~ 10도)
)

In [ ]:
train_data_dir = '/content/img_p/train'
val_data_dir = '/content/img_p/val'

image_size = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    zoom_range=0.2,
    preprocessing_function=lambda img: augment_face_pose(img, augmenter),  # 증강 함수 imgaug적용
)

val_datagen = ImageDataGenerator(rescale=1./255)

# 데이터 로드
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4324 images belonging to 4 classes.
Found 765 images belonging to 4 classes.


In [ ]:
train_data_dir = '/content/test_img_p/'

In [ ]:
import json

with open ('/content/drive/MyDrive/Project/Data/Test_data/label/test_anger.json', "r", encoding='euc-kr') as f:
    anger_label = json.load(f)
with open ('/content/drive/MyDrive/Project/Data/Test_data/label/test_happy.json', "r", encoding='euc-kr') as f:
    happy_label = json.load(f)
with open ('/content/drive/MyDrive/Project/Data/Test_data/label/test_panic.json', "r", encoding='euc-kr') as f:
    panic_label = json.load(f)
with open ('/content/drive/MyDrive/Project/Data/Test_data/label/test_sadness.json', "r", encoding='euc-kr') as f:
    sadness_label = json.load(f)

In [ ]:
def predict_emotion(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(image_size, image_size))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    class_labels = list(train_generator.class_indices.keys())
    return class_labels[predicted_class[0]]

def vitb16_predict_emotion(path):
  example_image_path = path
  predicted_emotion = predict_emotion(example_image_path)

  return predicted_emotion

def calcuate_true_accuracy(path, label, true_emotion):
  correct_count = 0
  uncorrect_count = 0
  error_count = 0

  for i in range(len(label)):
    try:
      predicted_emotion = vitb16_predict_emotion(path + label[i]['filename'])

      if true_emotion == predicted_emotion:
        correct_count += 1
      else:
        uncorrect_count += 1

      print(f'True Emotion    : {true_emotion}')
      print(f'Predict Emotion : {predicted_emotion}')
      print(f'{correct_count}, {uncorrect_count}, realtime acc : {correct_count/i:.2f}%')
    except Exception as e:
      error_count += 1
      continue

  return correct_count, uncorrect_count, error_count

In [ ]:
anger_cc, anger_uc, anger_ec = calcuate_true_accuracy('/content/test_img_p/anger/', anger_label, 'anger')
happy_cc, happy_uc, happy_ec = calcuate_true_accuracy('/content/test_img_p/happy/', happy_label, 'happy')
panic_cc, panic_uc, panic_ec = calcuate_true_accuracy('/content/test_img_p/panic/', panic_label, 'panic')
sadness_cc, sadness_uc, sadness_ec = calcuate_true_accuracy('/content/test_img_p/sadness/', sadness_label, 'sadness')

1/1 [==============================] - 0s 44ms/step
True Emotion    : anger
Predict Emotion : anger
1/1 [==============================] - 0s 41ms/step
True Emotion    : anger
Predict Emotion : anger
2, 0, realtime acc : 2.00%
1/1 [==============================] - 0s 43ms/step
True Emotion    : anger
Predict Emotion : sadness
2, 1, realtime acc : 1.00%
1/1 [==============================] - 0s 42ms/step
True Emotion    : anger
Predict Emotion : anger
3, 1, realtime acc : 1.00%
1/1 [==============================] - 0s 41ms/step
True Emotion    : anger
Predict Emotion : panic
3, 2, realtime acc : 0.75%
1/1 [==============================] - 0s 41ms/step
True Emotion    : anger
Predict Emotion : anger
4, 2, realtime acc : 0.80%
1/1 [==============================] - 0s 44ms/step
True Emotion    : anger
Predict Emotion : anger
5, 2, realtime acc : 0.83%
1/1 [==============================] - 0s 42ms/step
True Emotion    : anger
Predict Emotion : panic
5, 3, realtime acc : 0.71%
1/1 [====

In [ ]:
entire_correct_count = anger_cc + happy_cc + panic_cc + sadness_cc
entire_error_count = anger_ec + happy_ec + panic_ec + sadness_ec
entire_lable_length = len(anger_label) + len(happy_label) + len(panic_label) + len(sadness_label)
entire_emotion_accuracy = entire_correct_count / (entire_lable_length - entire_error_count) * 100

In [ ]:
# 예측을 위해 테스트 데이터 전처리를 시도했지만 전처리에 실패한 개수
print(f'Anger Error Image Count : {anger_ec}')
print(f'Happy Error Image Count : {happy_ec}')
print(f'Panic Error Image Count : {panic_ec}')
print(f'Sadness Error Image Count : {sadness_ec}')

Anger Error Image Count : 1
Happy Error Image Count : 1
Panic Error Image Count : 1
Sadness Error Image Count : 1


In [ ]:
# 예측 감정과 정답 감정이 일치한 개수
print(f'Anger Correct Count : {anger_cc}')
print(f'Happy Correct Count : {happy_cc}')
print(f'Panic Correct Count : {panic_cc}')
print(f'Sadness Correct Count : {sadness_cc}')

Anger Correct Count : 139
Happy Correct Count : 174
Panic Correct Count : 202
Sadness Correct Count : 113


In [ ]:
# 감정별 예측 정확도
print(f'Anger Predict Accuracy : {anger_cc / (len(anger_label) - anger_ec) * 100:.2f}%')
print(f'Happy Predict Accuracy : {happy_cc / (len(happy_label) - happy_ec) * 100:.2f}%')
print(f'Panic Predict Accuracy : {panic_cc / (len(panic_label) - panic_ec) * 100:.2f}%')
print(f'Sadness Predict Accuracy : {sadness_cc / (len(sadness_label) - sadness_ec) * 100:.2f}%')

# 전체 테스트 데이터셋에 대한 감정 예측 정확도
print(f'\nEntire Emotion Accuracy : {entire_emotion_accuracy:.2f}%')

Anger Predict Accuracy : 49.82%
Happy Predict Accuracy : 58.59%
Panic Predict Accuracy : 73.72%
Sadness Predict Accuracy : 39.93%

Entire Emotion Accuracy : 55.43%
